# Multilingual Named Entity Recognition

In [ ]:
!pip3 install datasets transformers matplotlib torch

Jusqu'à présent, dans le livre, nous avons appliqué des transformateurs pour résoudre des tâches NLP sur des corpus anglais, mais que faites-vous lorsque vos documents sont écrits en grec, `swahili` ou `klingon` ? Une approche consiste à rechercher dans le Hugging Face Hub un modèle de langage pré-entraîné approprié et à l'adapter à la tâche à accomplir. 

Cependant, ces modèles pré-entraînés ont tendance à n'exister que pour les langues à "hautes ressources" comme l'allemand, le russe ou le mandarin, où de nombreux textes web sont disponibles pour le pré-entraînement. 

Un autre défi commun se pose lorsque votre corpus est multilingue : maintenir de multiples modèles monolingues en production ne sera pas une partie de plaisir pour vous ou votre équipe d'ingénieurs.

Heureusement, il existe une classe de transformateurs multilingues qui viennent à la rescousse. Comme BERT, ces modèles utilisent la modélisation du langage masqué comme objectif de pré-entraînement, mais ils sont entraînés conjointement sur des textes dans plus de cent langues. En s'entraînant au préalable sur d'énormes corpus dans de nombreuses langues, ces transformateurs multilingues permettent un transfert interlinguistique " zéro coup ". Cela signifie qu'un modèle affiné dans une langue peut être appliqué à d'autres langues sans aucune formation supplémentaire ! Cela rend également ces modèles bien adaptés au "code-switching", où un locuteur alterne entre deux ou plusieurs langues ou dialectes dans le contexte d'une seule conversation.

Dans ce chapitre, nous allons explorer comment un modèle de transformation unique appelé `XLM-RoBERTa` (présenté au chapitre 3) peut être affiné pour effectuer la reconnaissance d'entités nommées (NER) dans plusieurs langues. 

Comme nous l'avons vu au chapitre 1, la reconnaissance d'entités nommées est une tâche courante de `TAL` qui identifie des entités telles que des personnes, des organisations ou des lieux dans un texte. Ces entités peuvent être utilisées pour diverses applications telles que l'obtention d'informations à partir de documents d'entreprise, l'amélioration de la qualité des moteurs de recherche ou simplement la création d'une base de données structurée à partir d'un corpus.

Dans le cadre de ce chapitre, supposons que nous souhaitons réaliser un `NER` pour un client basé en Suisse, où il existe quatre langues nationales (l'anglais servant souvent de passerelle entre elles). Commençons par obtenir un corpus multilingue adapté à ce problème.

Le `transfert zéro-shot` ou `l'apprentissage zéro-shot` fait généralement référence à la tâche consistant à former un modèle sur un ensemble d'étiquettes, puis à l'évaluer sur un autre ensemble d'étiquettes. Dans le contexte des transformateurs, l'apprentissage zéro-shot peut également se référer à des situations où un modèle de langage comme GPT-3 est évalué sur une tâche en aval sur laquelle il n'a même pas été affiné.

## Le jeu de données
Dans ce chapitre, nous utiliserons un sous-ensemble du benchmark Cross-lingual TRansfer Evaluation of Multilingual Encoders (XTREME) appelé WikiANN ou PAN-X.2 Ce jeu de données est constitué d'articles Wikipédia dans de nombreuses langues, dont les quatre langues les plus parlées en Suisse : L'allemand (62,9%), le français (22,9%), l'italien (8,4%) et l'anglais (5,9%). Chaque article est annoté avec les balises LOC (location), PER (person) et ORG (organization) dans le format "inside-outside-beginning" (IOB2). Dans ce format, un préfixe B- indique le début d'une entité, et des tokens consécutifs appartenant à la même entité reçoivent un préfixe I-. 
Une étiquette O indique que le token n'appartient à aucune entité. Par exemple, la phrase suivante :

Jeff Dean est un informaticien de Google en Californie.

Pour charger un des sous-ensembles `PAN-X` dans `XTREME`, nous devons savoir quelle configuration de jeu de données passer à la fonction `load_dataset()`. 
Lorsque vous avez affaire à un jeu de données comportant plusieurs domaines, vous pouvez utiliser la fonction `get_dataset_config_names()` pour savoir quels sous-ensembles sont disponibles :

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from datasets import get_dataset_config_names, load_dataset

In [16]:
from collections import defaultdict, Counter
from datasets import DatasetDict

In [18]:
from transformers import AutoTokenizer

In [3]:
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

XTREME has 183 configurations


In [4]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [6]:
load_dataset("xtreme", name="PAN-X.de")

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [8]:
langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

# Return a DatasetDict if a key doesn't exist
panx_ch = defaultdict(DatasetDict)

In [9]:
for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # Shuffle and downsample each split according to spoken proportion
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))

  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4. Subsequent calls will reuse this data.


...

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
pd.DataFrame({
    lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
    index=["Number of training examples"]
)

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


In [11]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [12]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [13]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


In [14]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names)

  0%|          | 0/12580 [00:00<?, ?ex/s]

  0%|          | 0/6290 [00:00<?, ?ex/s]

  0%|          | 0/6290 [00:00<?, ?ex/s]

In [15]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [17]:
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


Cela semble bon - les distributions des fréquences `PER`, `LOC` et `ORG` sont à peu près les mêmes pour chaque division, donc les ensembles de validation et de test devraient fournir une bonne mesure de la capacité de généralisation de notre marqueur `NER`. Ensuite, examinons quelques transformateurs multilingues populaires et la façon dont ils peuvent être adaptés pour s'attaquer à notre tâche de NER.

## Transformateurs multilingues
Les transformateurs multilingues impliquent des architectures et des procédures d'apprentissage similaires à celles de leurs homologues monolingues, sauf que le corpus utilisé pour le pré-entraînement est composé de documents dans de nombreuses langues. Une caractéristique remarquable de cette approche est qu'en dépit du fait qu'elle ne reçoit aucune information explicite pour différencier les langues, les représentations linguistiques résultantes sont capables de se généraliser à travers les langues pour une variété de tâches en aval. Dans certains cas, cette capacité à effectuer un transfert interlinguistique peut produire des résultats qui sont compétitifs avec ceux des modèles monolingues, ce qui permet de contourner la nécessité d'entraîner un modèle par langue !

Pour mesurer les progrès du transfert interlinguistique pour les NER, les jeux de données CoNLL-2002 et CoNLL-2003 sont souvent utilisés comme référence pour l'anglais, le néerlandais, l'espagnol et l'allemand. Ce référentiel est constitué d'articles de presse annotés avec les mêmes catégories LOC, PER et ORG que PAN-X, mais il contient une étiquette supplémentaire MISC pour les entités diverses qui n'appartiennent pas aux trois groupes précédents. Les modèles de transformateurs multilingues sont généralement évalués de trois manières différentes :

- `en`
  Fine-tune on the English training data and then evaluate on each language’s test set.

- `each`
  Fine-tune and evaluate on monolingual test data to measure per-language performance.

- `all`
  Fine-tune on all the training data to evaluate on all on each language’s test set.

Nous adopterons une stratégie d'évaluation similaire pour notre tâche NER, mais nous devons d'abord sélectionner un modèle à évaluer. L'un des premiers transformateurs multilingues était mBERT, qui utilise la même architecture et le même objectif de pré-entraînement que BERT, mais ajoute des articles de Wikipédia de plusieurs langues au corpus de pré-entraînement. Depuis, `mBERT` a été remplacé par `XLM-RoBERTa` (ou `XLM-R` en abrégé), c'est donc ce modèle que nous allons considérer dans ce chapitre.

Comme nous l'avons vu au chapitre 3, `XLM-R` utilise uniquement la MLM comme objectif de pré-entraînement pour 100 langues, mais se distingue par la taille énorme de son corpus de pré-entraînement par rapport à ses prédécesseurs : Des vidages de Wikipedia pour chaque langue et 2,5 téraoctets de données Common Crawl provenant du web. Ce corpus est plusieurs ordres de grandeur plus grand que ceux utilisés dans les modèles précédents et fournit une augmentation significative du signal pour les langues à faibles ressources comme le birman et le swahili, où seul un petit nombre d'articles de Wikipedia existe.

La partie `RoBERTa` du nom du modèle fait référence au fait que l'approche de pré-entraînement est la même que pour les modèles monolingues `RoBERTa`. 

Les développeurs de `RoBERTa` ont amélioré plusieurs aspects de `BERT`, en particulier en supprimant complètement la tâche de prédiction de la phrase suivante.

`XLM-R` abandonne également les encastrements de langue utilisés dans XLM et utilise `SentencePiece` pour tokeniser directement les textes bruts.Outre sa nature multilingue, une différence notable entre `XLM-R` et `RoBERTa` est la taille des vocabulaires respectifs : 250.000 tokens contre 55.000 !

`XLM-R` est un excellent choix pour les tâches NLU multilingues. Dans la prochaine section, nous allons explorer comment il peut efficacement tokeniser à travers de nombreuses langues.

In [19]:
bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [20]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [21]:
xlmr_tokens

['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']

In [22]:
bert_tokens

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']

Nous voyons ici qu'au lieu des jetons `[CLS]` et `[SEP]` que BERT utilise pour les tâches de classification de phrases, `XLM-R` utilise `<s>` et `<\s>` pour indiquer le début et la fin d'une séquence. Ces jetons sont ajoutés lors de l'étape finale de la tokenisation, comme nous le verrons ensuite.

### Le pipeline de tokenisation
Jusqu'à présent, nous avons traité la tokenisation comme une opération unique qui transforme les chaînes de caractères en entiers que nous pouvons faire passer dans le modèle. Ce n'est pas tout à fait exact, et si nous regardons de plus près, nous pouvons voir qu'il s'agit en fait d'un pipeline de traitement complet qui se compose généralement de quatre étapes

- `Normalization`
- `PreTokenization`
- `Tokenize Model`
- `Postprocessing`

Examinons de plus près chaque étape du traitement et illustrons leur effet à l'aide de la phrase d'exemple "Jack Sparrow aime New York !":

- `Normalisation`
Cette étape correspond à l'ensemble des opérations que vous appliquez à une chaîne brute pour la rendre plus "propre". Les opérations courantes consistent à supprimer les espaces blancs et les caractères accentués. La normalisation Unicode est une autre opération de normalisation courante appliquée par de nombreux tokenizers pour faire face au fait qu'il existe souvent plusieurs façons d'écrire le même caractère. Cela peut faire que deux versions de la "même" chaîne (c'est-à-dire avec la même séquence de caractères abstraits) apparaissent différentes ; les schémas de normalisation Unicode comme NFC, NFD, NFKC et NFKD remplacent les différentes façons d'écrire le même caractère par des formes standard. Un autre exemple de normalisation est la mise en minuscule. Si le modèle est censé n'accepter et n'utiliser que des caractères minuscules, cette technique peut être utilisée pour réduire la taille du vocabulaire dont il a besoin. Après normalisation, notre exemple de chaîne de caractères ressemblerait à "jack sparrow loves new york !".

- `Pretokenization`
Cette étape divise un texte en objets plus petits qui donnent une limite supérieure à ce que seront vos tokens à la fin de la formation. Une bonne façon de voir les choses est que le prétokéniseur divise votre texte en "mots", et que vos jetons finaux seront des parties de ces mots. Pour les langues qui le permettent (l'anglais, l'allemand et de nombreuses langues indo-européennes), les chaînes de caractères peuvent généralement être divisées en mots sur les espaces blancs et la ponctuation. Par exemple, cette étape peut transformer nos ["jack", "sparrow", "loves", "new", "york", " !"]. Ces mots sont ensuite plus simples à diviser en sous-mots à l'aide des algorithmes BPE (Byte-Pair Encoding) ou Unigram dans l'étape suivante du pipeline. Cependant, la division en "mots" n'est pas toujours une opération triviale et déterministe, ni même une opération qui a du sens. Par exemple, dans des langues comme le chinois, le japonais ou le coréen, le regroupement des symboles en unités sémantiques comme les mots indo-européens peut être une opération non déterministe avec plusieurs groupes également valables. Dans ce cas, il peut être préférable de ne pas prétokéniser le texte et d'utiliser une bibliothèque spécifique à la langue pour la prétokénisation.

- `Tokenize Model`
Une fois les textes d'entrée normalisés et prétokénisés, le tokenizer applique un modèle de découpage en sous-mots sur les mots. C'est la partie du pipeline qui doit être entraînée sur votre corpus (ou qui a été entraînée si vous utilisez un tokenizer pré-entraîné). Le rôle du modèle est de diviser les mots en sous-mots pour réduire la taille du vocabulaire et essayer de réduire le nombre de tokens hors vocabulaire. Plusieurs algorithmes de tokenisation de sous-mots existent, notamment BPE, Unigram et WordPiece. Par exemple, notre exemple courant pourrait ressembler à [jack, spa, rrow, loves, new, york, !] après l'application du modèle de tokenisation. Notez qu'à ce stade, nous n'avons plus une liste de chaînes de caractères mais une liste d'entiers (ID d'entrée) ; pour que l'exemple reste illustratif, nous avons conservé les mots mais supprimé les guillemets pour indiquer la transformation.

- `Postprocessing`
Il s'agit de la dernière étape du pipeline de tokénisation, au cours de laquelle certaines transformations supplémentaires peuvent être appliquées à la liste de tokens - par exemple, l'ajout de tokens spéciaux au début ou à la fin de la séquence d'index de tokens en entrée. Par exemple, un tokeniseur de type BERT ajouterait des classifications et des séparateurs : [CLS, jack, spa, rrow, loves, new, york, !, SEP]. Cette séquence (rappelez-vous qu'il s'agira d'une séquence d'entiers, et non des tokens que vous voyez ici) peut ensuite être introduite dans le modèle.



### Le tokeniseur SentencePiece
Le tokenizer de SentencePiece est basé sur un type de segmentation des sous-mots appelé Unigram et encode chaque texte d'entrée comme une séquence de caractères Unicode. Cette dernière caractéristique est particulièrement utile pour les corpus multilingues car elle permet à SentencePiece d'être agnostique vis-à-vis des accents, de la ponctuation et du fait que de nombreuses langues, comme le japonais, n'ont pas de caractères d'espacement. Une autre particularité de SentencePiece est que les espaces blancs se voient attribuer le symbole Unicode U+2581, ou le caractère ▁, également appelé caractère de bloc d'un quart inférieur. Cela permet à SentencePiece de détokéniser une séquence sans ambiguïté et sans s'appuyer sur des pré-tokéniseurs spécifiques à la langue. Dans notre exemple de la section précédente, par exemple, nous pouvons voir que WordPiece a perdu l'information selon laquelle il n'y a pas d'espace entre "York" et " !". En revanche, SentencePiece préserve les espaces dans le texte tokénisé, de sorte que nous pouvons reconvertir le texte brut sans ambiguïté :

In [23]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

'<s> Jack Sparrow loves New York!</s>'